In [1]:
from utils.ModelWrapper import ModelWrapper
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
import wandb

In [2]:
# Init
wandb_project_name = 'deepfake-basic'
data_dir = "./data/data-large/data"
img_size = 128
wandb.init(project=wandb_project_name, entity="dat550")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dat550 (use `wandb login --relogin` to force relogin)


### Wandb Sweep training of the model

In [3]:
def train():
   # Default values for hyper-parameters we're going to sweep over
   config_defaults = {
         'epochs': 15,
         'batch_size': 32,
         'learning_rate': 0.0001,
         'optimizer': 'nadam',
         'hidden_layer_size': 64,
         'conv_layer_1_size': 16,
         'conv_layer_2_size': 32,
         'conv_layer_3_size': 64,
         'dropout': 0.5,
         "use_augmentation": True,
      }
   # Initialize wandb with a sample project name
   wandb.init(config=config_defaults, project=wandb_project_name, entity="dat550")

   # # Config is a variable that holds and saves hyperparameters and inputs
   config = wandb.config

   # init wrapper
   model_wrapper = ModelWrapper(data_dir, img_size, config.batch_size,  use_generator=False, sample_datasets=True)

   data_augmentation = Sequential(
      [
         layers.RandomFlip("horizontal", input_shape=(img_size, img_size, 3)),
         layers.RandomRotation(0.1),
         layers.RandomZoom(0.1),
      ]
   )

   input_layers = [
      data_augmentation if config.use_augmentation else None,
      layers.Conv2D(filters = config.conv_layer_1_size, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_size, img_size, 3)),
      layers.MaxPooling2D(),
      layers.Conv2D(filters = config.conv_layer_2_size, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(filters = config.conv_layer_3_size, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(config.dropout) if config.dropout > 0 else None,
      layers.Flatten(),
      layers.Dense(config.hidden_layer_size, activation='relu'),
      layers.Dense(units = 1, activation='sigmoid')
   ]
   input_layers = [e for e in input_layers if e is not None]

   # create model
   model_wrapper.create_model(None, input_layers, config)

   # Define the optimizer
   if config.optimizer=='sgd':
      optimizer = tf.keras.optimizers.SGD(learning_rate=config.learning_rate) 
   elif config.optimizer=='rmsprop':
      optimizer = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate,)
   elif config.optimizer=='adam':
      optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate,) 
   elif config.optimizer=='nadam':
      optimizer = tf.keras.optimizers.Nadam(learning_rate=config.learning_rate,) 
   
   # Compile the model
   model_wrapper.model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
   # Train the model
   model_wrapper.fit()

In [5]:
# sweep training
sweep_config = {
    "name": "deepfake-basic-sweep",
   'method': 'bayes',
    "metric": {
            "name": "accuracy",
            "goal": "maximize"
        },
    "early_terminate": {
       "type": "hyperband",
       "min_iter": 5
   },
   'parameters': {
        'batch_size': {
            'values': [64, 128, 256] 
        },
        'dropout': {
            'values': [0.0, 0.2, 0.5]
        },
        'conv_layer_1_size': {
            'values': [16,32], #32
        },
        'conv_layer_2_size': {
            'values': [32,64] #64
        },
        'conv_layer_3_size': {
            'values': [64 ,128] # 128
        },
        'hidden_layer_size': {
                'values': [64, 128] 
            },
      'learning_rate': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4]
        },
        'optimizer': {
            'values': ['adam',  "nadam", 'sgd', 'rmsprop']
        },
          "use_augmentation": {
            "values": [False, True] 
        }
   }
}
sweep_id = wandb.sweep(sweep_config, project=wandb_project_name, entity="dat550")
wandb.agent(sweep_id, function=train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 05mt6dlf
Sweep URL: https://wandb.ai/dat550/deepfake-basic/sweeps/05mt6dlf


wandb: Agent Starting Run: gsoyhk76 with config:
wandb: 	batch_size: 256
wandb: 	conv_layer_1_size: 16
wandb: 	conv_layer_2_size: 64
wandb: 	conv_layer_3_size: 128
wandb: 	dropout: 0.2
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: sgd
wandb: 	use_augmentation: False
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 29988 files belonging to 2 classes.
Found 3431 files belonging to 2 classes.
Found 1587 files belonging to 2 classes.


wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


...

wandb: Agent Starting Run: zlgp2pbz with config:
wandb: 	batch_size: 128
wandb: 	conv_layer_1_size: 32
wandb: 	conv_layer_2_size: 64
wandb: 	conv_layer_3_size: 64
wandb: 	dropout: 0
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: rmsprop
wandb: 	use_augmentation: True
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 29988 files belonging to 2 classes.
Found 3431 files belonging to 2 classes.
Found 1587 files belonging to 2 classes.
235/235 [==============================] - 449s 2s/step - loss: 2.6025 - accuracy: 0.5720 - val_loss: 0.6449 - val_accuracy: 0.6284 - _timestamp: 1652509948.0000 - _runtime: 462.0000



accuracy,▁
epoch,▁
loss,▁
val_accuracy,▁
val_loss,▁
accuracy,0.57203
best_epoch,0
best_val_loss,0.64493
epoch,0
loss,2.6025
val_accuracy,0.62839


wandb: Agent Starting Run: r5uj57s4 with config:
wandb: 	batch_size: 64
wandb: 	conv_layer_1_size: 16
wandb: 	conv_layer_2_size: 64
wandb: 	conv_layer_3_size: 128
wandb: 	dropout: 0
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.0003
wandb: 	optimizer: rmsprop
wandb: 	use_augmentation: True
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 29988 files belonging to 2 classes.
Found 3431 files belonging to 2 classes.
Found 1587 files belonging to 2 classes.
372/469 [======================>.......] - ETA: 1:28 - loss: 1.6700 - accuracy: 0.5946

### Evaluate model

In [ ]:
# load the bess model NB: edit namd and path
loaded_model = wandb.restore('model-best.h5', run_path="dat550/deepfake-efficientnet/runs/29425ckm")
best_model = load_model(loaded_model.name)

In [ ]:
model_wrapper = ModelWrapper(data_dir, img_size, 32)
model_wrapper.evaluate_model(best_model)

In [ ]:
model_wrapper.export_to_png()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
